<a href="https://colab.research.google.com/github/divsal009/div/blob/master/1_HOMEAI_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn pyngrok pandas fuzzywuzzy transformers nest_asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.7 MB/s eta 0:00:00


In [2]:
!pip install transformers

In [3]:
!pip install fuzzywuzzy


In [4]:
!pip install flask flask-ngrok fuzzywuzzy transformers


In [5]:
!pip install pyngrok


In [6]:
!pip install flask flask-ngrok pandas fuzzywuzzy transformers


In [7]:
!pip install fastapi uvicorn nest_asyncio pyngrok


In [8]:
!pip install python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.4 MB/s eta 0:00:00


In [9]:
!ngrok config add-authtoken 2pi17RYPIkF3FyQFq4LL2dKzZCi_7c8B2Cxa8bzdg8MK8cayd


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
!pip install python-multipart


In [11]:
!pip install transformers fastapi uvicorn pandas fuzzywuzzy


In [12]:
!pip install nest_asyncio


In [13]:
import nest_asyncio
nest_asyncio.apply()


In [14]:
!pip install pyngrok


In [ ]:
import os
import pandas as pd
from fuzzywuzzy import fuzz, process
from fastapi import FastAPI, Form, Request
from fastapi.responses import HTMLResponse
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import re
from transformers import pipeline

# Apply nest_asyncio for Colab compatibility
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Load the Excel data
file_path = "/content/Datasetfinaldivya12_11.xlsx"
if not os.path.exists(file_path):
    raise FileNotFoundError("The dataset file was not found at the specified path.")

excel_data = pd.ExcelFile(file_path)
disease_bodyfn_df = excel_data.parse('30-10-DISEASE VS BODYFN')
bodyfn_assistive_df = excel_data.parse('31-10 BODYFN vs assistive techn')
bodyfn_home_icf_df = excel_data.parse('31-10 BODYFN vs HOME ICF')

# Preprocessing helper function
def preprocess_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text.strip().lower()) if text else ""

# Normalize the dataset
disease_bodyfn_df['Disease'] = disease_bodyfn_df['Disease'].apply(preprocess_text)
bodyfn_assistive_df['Symptoms'] = bodyfn_assistive_df['Symptoms'].apply(preprocess_text)
bodyfn_home_icf_df['Symptoms'] = bodyfn_home_icf_df['Symptoms'].apply(preprocess_text)

# Fuzzy Matching Helper Function with prioritization
def find_closest_match(input_text, text_list, threshold=85):
    input_text = preprocess_text(input_text)

    # Attempt exact or near-exact match first
    if input_text in text_list:
        return input_text

    # Use fuzzy matching as fallback
    match = process.extractOne(input_text, text_list, scorer=fuzz.token_sort_ratio)
    return match[0] if match and match[1] >= threshold else None

# Enhanced Keyword Extraction with priority on exact matches
def extract_keyword(input_text):
    input_text = preprocess_text(input_text)
    tokens = input_text.split()

    # Create a unified list of potential matches (disease + symptoms)
    all_keywords = set(
        disease_bodyfn_df['Disease'].tolist()
        + bodyfn_assistive_df['Symptoms'].tolist()
        + bodyfn_home_icf_df['Symptoms'].tolist()
    )

    # Step 1: Check for exact matches first
    exact_matches = [kw for kw in all_keywords if kw in input_text]
    if exact_matches:
        return exact_matches[0]

    # Step 2: Token-based matching
    for token in tokens:
        match = find_closest_match(token, list(all_keywords), threshold=85)  # Use a high threshold
        if match:
            return match

    # Step 3: Fallback to fuzzy match for the entire input
    match = find_closest_match(input_text, list(all_keywords), threshold=75)  # Lower threshold for fallback
    return match

# Initialize Hugging Face LLM
generator = pipeline("text-generation", model="distilgpt2")

# Generate conversational response
def generate_conversational_response(keyword, assistive_tech, home_adaptations):
    prompt = (
        f"Provide a detailed and user-friendly explanation about '{keyword}'. "
        f"Assistive technologies include: {assistive_tech}. "
        f"Home adaptations include: {home_adaptations}."
    )
    response = generator(
        prompt,
        max_length=100,
        num_return_sequences=1,
        truncation=True,
    )[0]["generated_text"]
    clean_response = response.split(".")[0] + "."
    return clean_response

# Navbar and styles
def navbar_and_styles():
    return """
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f0f8ff;
            margin: 0;
            padding: 0;
        }
        .navbar {
            background-color: #004d00;
            padding: 15px;
            text-align: center;
        }
        .navbar a {
            color: white;
            text-decoration: none;
            font-size: 18px;
            margin: 15px;
        }
        .navbar a:hover {
            text-decoration: underline;
        }
        .container {
            max-width: 800px;
            margin: 50px auto;
            text-align: left;
            padding: 20px;
            background-color: #ffffff;
            border-radius: 10px;
            box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
        }
        h1, h2 {
            color: #004d00;
            margin-bottom: 20px;
        }
        p {
            color: #333;
            font-size: 18px;
            line-height: 1.8;
        }
        input[type="text"] {
            width: 100%;
            padding: 10px;
            font-size: 18px;
            margin: 10px 0;
        }
        button {
            padding: 10px 20px;
            font-size: 18px;
            background-color: #004d00;
            color: white;
            border: none;
            border-radius: 5px;
        }
        button:hover {
            background-color: #003300;
        }
        .result-container {
            padding: 20px;
            background-color: #f9f9f9;
            border-radius: 10px;
            box-shadow: 0 2px 6px rgba(0, 0, 0, 0.1);
        }
        .result-container h2 {
            color: #004d00;
        }
    </style>
    <div class='navbar'>
        <a href="/">Home</a>
        <a href="/disease">Disease/Symptom Search</a>
        <a href="/contact">Contact Us</a>
    </div>
    """

# Home Page
@app.get("/", response_class=HTMLResponse)
async def home():
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>HOMEAI-ENABLE</title>
        {navbar_and_styles()}
    </head>
    <body>
        <div class="container">
            <h1>Welcome to HOMEAI-ENABLE</h1>
            <p>Your trusted platform for insights into diseases, symptoms, assistive technologies, and personalized home adaptations.</p>
            <p>Start by entering a disease or symptom, and let us guide you toward solutions that enhance your well-being.</p>
        </div>
    </body>
    </html>
    """

@app.get("/disease", response_class=HTMLResponse)
async def disease_get():
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Disease Search</title>
        {navbar_and_styles()}
    </head>
    <body>
        <div class="container">
            <h1>Search for Diseases or Symptoms</h1>
            <form method="post" action="/disease">
                <input type="text" name="disease_name" placeholder="Enter a disease or symptom" required>
                <button type="submit" style="margin-top: 15px;">Search</button>
            </form>
        </div>
    </body>
    </html>
    """

@app.post("/disease", response_class=HTMLResponse)
async def disease_post(disease_name: str = Form(...)):
    keyword = extract_keyword(disease_name)
    response = navbar_and_styles()
    response += "<div class='container'>"

    if not keyword:
        response += f"<h1 style='font-size: 20px;'>Sorry, we couldn't find any information for '{disease_name}'.</h1></div>"
        return HTMLResponse(content=response)

    matched_disease = find_closest_match(keyword, disease_bodyfn_df['Disease'].tolist())
    matched_symptom = find_closest_match(keyword, bodyfn_assistive_df['Symptoms'].tolist() + bodyfn_home_icf_df['Symptoms'].tolist())

    if matched_disease:
        disease_info = disease_bodyfn_df[disease_bodyfn_df['Disease'] == matched_disease]
        symptoms = disease_info['Symptoms'].values[0].split(", ") if not disease_info.empty else []
        response += f"<h1>Here's what I found about the disease '{matched_disease.capitalize()}':</h1>"
        for symptom in symptoms:
            assistive_tech = "No assistive technology found"
            home_adaptations = "No home adaptations found"

            tech_info = bodyfn_assistive_df[bodyfn_assistive_df['Symptoms'] == preprocess_text(symptom)]
            home_info = bodyfn_home_icf_df[bodyfn_home_icf_df['Symptoms'] == preprocess_text(symptom)]

            if not tech_info.empty:
                assistive_tech = tech_info['Assistive Technology'].values[0]
            if not home_info.empty:
                home_adaptations = ", ".join(home_info['Home functions'].dropna().tolist())

            response += f"<div class='result-container'><h2>Symptom: {symptom.capitalize()}</h2><p><strong>Assistive Technologies:</strong> {assistive_tech}</p><p><strong>Home Functions:</strong> {home_adaptations}</p></div>"

    if matched_symptom:
        assistive_info = bodyfn_assistive_df[bodyfn_assistive_df['Symptoms'] == matched_symptom]
        home_info = bodyfn_home_icf_df[bodyfn_home_icf_df['Symptoms'] == matched_symptom]

        assistive_tech = assistive_info['Assistive Technology'].values[0] if not assistive_info.empty else "No assistive technology found"
        home_adaptations = ", ".join(home_info['Home functions'].dropna().tolist()) if not home_info.empty else "No home adaptations found"

        response += f"<div class='result-container'><h2>Symptom: {matched_symptom.capitalize()}</h2><p><strong>Assistive Technologies:</strong> {assistive_tech}</p><p><strong>Home Functions:</strong> {home_adaptations}</p></div>"

    response += "</div>"
    return HTMLResponse(content=response)

# Public URL and run command (use when deploying locally or publicly)
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
uvicorn.run(app, host="127.0.0.1", port=8000)


Public URL: NgrokTunnel: "https://e607-34-169-38-251.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [233]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     81.78.214.135:0 - "GET / HTTP/1.1" 200 OK
INFO:     81.78.214.135:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     81.78.214.135:0 - "GET /disease HTTP/1.1" 200 OK
INFO:     81.78.214.135:0 - "POST /disease HTTP/1.1" 200 OK
INFO:     81.78.214.135:0 - "GET /disease HTTP/1.1" 200 OK
INFO:     81.78.214.135:0 - "POST /disease HTTP/1.1" 200 OK
